In [19]:
import sqlite3


In [20]:
import csv
import tkinter as tk
from tkinter import ttk, messagebox


In [21]:
import logging
import csv
from datetime import datetime


In [22]:
import joblib

model = joblib.load("spam_model.pkl")
vectorizer = joblib.load("tfidf_vectorizer.pkl")


In [23]:
conn = sqlite3.connect("spam_system.db")
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS users (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    username TEXT UNIQUE,
    password TEXT,
    role TEXT
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS history (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    username TEXT,
    message TEXT,
    prediction TEXT,
    probability TEXT,
    timestamp TEXT
)
""")

conn.commit()


In [24]:
current_user = None
current_role = None


In [25]:
def register_user():
    username = reg_username.get()
    password = reg_password.get()

    if username == "" or password == "":
        messagebox.showerror("Error", "All fields required")
        return

    try:
        cursor.execute(
            "INSERT INTO users (username, password, role) VALUES (?, ?, ?)",
            (username, password, "user")
        )
        conn.commit()
        messagebox.showinfo("Success", "Registration successful")
        register_window.destroy()
    except sqlite3.IntegrityError:
        messagebox.showerror("Error", "Username already exists")


In [26]:
def open_register():
    global register_window, reg_username, reg_password

    register_window = tk.Toplevel(login_window)
    register_window.title("Register")
    register_window.geometry("300x220")

    tk.Label(register_window, text="Register", font=("Arial", 14, "bold")).pack(pady=10)

    tk.Label(register_window, text="Username").pack()
    reg_username = tk.Entry(register_window)
    reg_username.pack()

    tk.Label(register_window, text="Password").pack()
    reg_password = tk.Entry(register_window, show="*")
    reg_password.pack()

    tk.Button(
        register_window,
        text="Register",
        command=register_user,
        bg="#2196F3",
        fg="white"
    ).pack(pady=15)


In [27]:
# ================== USERS & ROLES ==================
USERS = {
    "admin": {"password": "admin123", "role": "admin"},
    "user1": {"password": "user123", "role": "user"}
}

current_user = None
current_role = None


In [28]:
logging.basicConfig(
    filename="app.log",
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

logging.info("SMS Spam Detection Application Started")

In [29]:
HISTORY_FILE = "prediction_history.csv"

# Create file with header if it does not exist
try:
    with open(HISTORY_FILE, "x", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["Timestamp", "Message", "Prediction", "Probability"])
except FileExistsError:
    pass


In [30]:
import os
os.getcwd()


'C:\\Users\\Zain\\Downloads\\Text_Classification_Project\\Text_Classification_Project'

In [31]:
import joblib

model = joblib.load("spam_model.pkl")
vectorizer = joblib.load("tfidf_vectorizer.pkl")

print("✅ Model and vectorizer loaded successfully")


✅ Model and vectorizer loaded successfully


In [32]:
# ================== IMPORTS ==================
import tkinter as tk
from tkinter import ttk, messagebox
import joblib
import sqlite3
import logging
from datetime import datetime

# ================== LOAD MODEL ==================
model = joblib.load("spam_model.pkl")
vectorizer = joblib.load("tfidf_vectorizer.pkl")

# ================== LOGGING ==================
logging.basicConfig(
    filename="app.log",
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    encoding="utf-8"
)

# ================== DATABASE ==================
conn = sqlite3.connect("spam_system.db")
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS users(
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    username TEXT UNIQUE,
    password TEXT,
    role TEXT
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS history(
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    username TEXT,
    message TEXT,
    prediction TEXT,
    probability TEXT,
    timestamp TEXT
)
""")

cursor.execute("""
INSERT OR IGNORE INTO users(username,password,role)
VALUES('admin','admin123','admin')
""")

conn.commit()

# ================== GLOBAL STATE ==================
current_user = None
current_role = None
theme = "dark"

# ================== COLORS ==================
DARK_BG = "#121212"
DARK_FG = "#ffffff"
LIGHT_BG = "#f5f5f5"
LIGHT_FG = "#000000"

# ================== THEME ==================
def apply_theme(win):
    bg = DARK_BG if theme == "dark" else LIGHT_BG
    fg = DARK_FG if theme == "dark" else LIGHT_FG
    win.configure(bg=bg)
    return bg, fg

def toggle_theme(win):
    global theme
    theme = "light" if theme == "dark" else "dark"
    win.destroy()
    open_login()

# ================== LOGIN ==================
def login():
    global current_user, current_role
    u = username_entry.get()
    p = password_entry.get()

    cursor.execute(
        "SELECT role FROM users WHERE username=? AND password=?",
        (u, p)
    )
    res = cursor.fetchone()

    if res:
        current_user = u
        current_role = res[0]
        login_win.destroy()
        open_main()
    else:
        messagebox.showerror("Login Failed", "Invalid username or password")

# ================== REGISTER ==================
def open_register():
    reg = tk.Toplevel(login_win)
    reg.title("Register")
    bg, fg = apply_theme(reg)
    reg.geometry("300x260")

    tk.Label(reg, text="Register", font=("Arial", 14, "bold"), bg=bg, fg=fg).pack(pady=10)
    u = tk.Entry(reg); u.pack(pady=5)
    p = tk.Entry(reg, show="*"); p.pack(pady=5)

    def save():
        try:
            cursor.execute(
                "INSERT INTO users(username,password,role) VALUES(?,?,?)",
                (u.get(), p.get(), "user")
            )
            conn.commit()
            messagebox.showinfo("Success", "Registered successfully")
            reg.destroy()
        except sqlite3.IntegrityError:
            messagebox.showerror("Error", "Username already exists")

    tk.Button(reg, text="Register", command=save, bg="#2196F3", fg="white").pack(pady=10)

# ================== FORGOT PASSWORD ==================
def forgot_password():
    win = tk.Toplevel(login_win)
    win.title("Forgot Password")
    bg, fg = apply_theme(win)
    win.geometry("300x220")

    tk.Label(win, text="Reset Password", font=("Arial", 14, "bold"), bg=bg, fg=fg).pack(pady=10)
    u = tk.Entry(win); u.pack(pady=5)
    p = tk.Entry(win, show="*"); p.pack(pady=5)

    def reset():
        cursor.execute(
            "UPDATE users SET password=? WHERE username=?",
            (p.get(), u.get())
        )
        if cursor.rowcount == 0:
            messagebox.showerror("Error", "User not found")
        else:
            conn.commit()
            messagebox.showinfo("Success", "Password updated")
            win.destroy()

    tk.Button(win, text="Reset", command=reset, bg="#4CAF50", fg="white").pack(pady=10)

# ================== LOGIN WINDOW ==================
def open_login():
    global login_win, username_entry, password_entry

    login_win = tk.Tk()
    login_win.title("SMS Spam Detection - Login")
    login_win.geometry("360x440")
    login_win.resizable(False, False)

    bg, fg = apply_theme(login_win)

    container = tk.Frame(login_win, bg=bg)
    container.pack(expand=True)

    tk.Label(container, text="🔐 Login", font=("Arial", 18, "bold"), bg=bg, fg=fg).pack(pady=(20, 25))

    tk.Label(container, text="Username", bg=bg, fg=fg).pack(anchor="w", padx=40)
    username_entry = tk.Entry(container, width=30)
    username_entry.pack(pady=6, padx=40)

    tk.Label(container, text="Password", bg=bg, fg=fg).pack(anchor="w", padx=40)
    password_entry = tk.Entry(container, width=30, show="*")
    password_entry.pack(pady=6, padx=40)

    tk.Button(container, text="Login", command=login,
              bg="#4CAF50", fg="white", width=25).pack(pady=(20, 8))

    tk.Button(container, text="Register", command=open_register,
              bg="#2196F3", fg="white", width=25).pack(pady=5)

    tk.Button(container, text="Forgot Password?",
              command=forgot_password, bg=bg, fg="#2196F3", bd=0).pack(pady=10)

    bottom = tk.Frame(login_win, bg=bg)
    bottom.pack(fill="x", pady=10)

    tk.Button(bottom, text="🌙 / ☀ Theme",
              command=lambda: toggle_theme(login_win),
              bg=bg, fg=fg, bd=0).pack(side="left", padx=20)

    tk.Button(bottom, text="Cancel",
              command=login_win.destroy,
              bg="#9E9E9E").pack(side="right", padx=20)

    tk.Label(login_win,
             text="SMS Spam Detection System | NLP + ML",
             font=("Arial", 9), bg=bg, fg="gray").pack(side="bottom", pady=10)

    login_win.mainloop()

# ================== MAIN APP ==================
def open_main():
    main = tk.Tk()
    bg, fg = apply_theme(main)
    main.title("SMS Spam Detection System")
    main.geometry("700x560")

    tk.Label(
        main,
        text=f"SMS Spam Detection\nUser: {current_user} ({current_role.upper()})",
        font=("Arial", 16, "bold"),
        bg=bg,
        fg=fg
    ).pack(pady=10)

    text = tk.Text(main, height=6, width=70, bg=bg, fg=fg, insertbackground=fg)
    text.pack(pady=10)

    result_lbl = tk.Label(main, text="", font=("Arial", 14, "bold"), bg=bg, fg=fg)
    result_lbl.pack(pady=10)

    def predict():
        msg = text.get("1.0", tk.END).strip()
        if not msg:
            messagebox.showwarning("Input Error", "Enter an SMS")
            return

        vec = vectorizer.transform([msg])
        pred = model.predict(vec)[0]
        prob = model.predict_proba(vec)[0][pred] * 100

        label = "SPAM 🚨" if pred == 1 else "HAM ✅"
        color = "#b71c1c" if pred == 1 else "#1b5e20"

        result_lbl.config(text=f"{label} ({prob:.2f}%)", bg=color, fg="white")

        cursor.execute("""
            INSERT INTO history(username,message,prediction,probability,timestamp)
            VALUES(?,?,?,?,?)
        """, (
            current_user,
            msg,
            "SPAM" if pred == 1 else "HAM",
            f"{prob:.2f}%",
            datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        ))
        conn.commit()

        logging.info(f"{current_user} | {label} | {prob:.2f}% | {msg}")

    def view_history():
        win = tk.Toplevel(main)
        win.title("Prediction History")
        win.geometry("900x400")

        cols = ("User", "Message", "Prediction", "Probability", "Time")
        tree = ttk.Treeview(win, columns=cols, show="headings")

        for c in cols:
            tree.heading(c, text=c)

        tree.column("User", width=100)
        tree.column("Message", width=350)
        tree.column("Prediction", width=120)
        tree.column("Probability", width=120)
        tree.column("Time", width=160)

        tree.pack(fill=tk.BOTH, expand=True)

        if current_role == "admin":
            cursor.execute(
                "SELECT username,message,prediction,probability,timestamp FROM history"
            )
        else:
            cursor.execute(
                "SELECT username,message,prediction,probability,timestamp FROM history WHERE username=?",
                (current_user,)
            )

        rows = cursor.fetchall()
        if not rows:
            messagebox.showinfo("Info", "No history found")
        for r in rows:
            tree.insert("", tk.END, values=r)

    def view_users():
        if current_role != "admin":
            return
        win = tk.Toplevel(main)
        win.title("Users (Admin)")
        win.geometry("400x300")

        tree = ttk.Treeview(win, columns=("Username", "Password", "Role"), show="headings")
        for c in ("Username", "Password", "Role"):
            tree.heading(c, text=c)
        tree.pack(fill=tk.BOTH, expand=True)

        cursor.execute("SELECT username,password,role FROM users")
        for r in cursor.fetchall():
            tree.insert("", tk.END, values=r)

    def logout():
        main.destroy()
        open_login()

    tk.Button(main, text="Predict", command=predict,
              width=15, bg="#4CAF50", fg="white").pack(pady=5)

    tk.Button(main, text="View History",
              command=view_history, width=20).pack(pady=5)

    if current_role == "admin":
        tk.Button(main, text="View Users",
                  command=view_users, width=20).pack(pady=5)

    tk.Button(main, text="Logout",
              command=logout, bg="#9E9E9E", width=20).pack(pady=15)

    main.mainloop()

# ================== START ==================
open_login()
